In [1]:
import openseespy.opensees as ops
import numpy as np
import opsvis as opsv
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Units
m = 1
N = 1
Pa = 1

inches = 0.0254 * m
ft = 12 * inches
kip = 4.45 * 10**3 * N
ksi = 6.89 * 10**6 * Pa

# Input variables

# Node Coordinates
x1 = 0.
y1 = 0.

x2 = widgets.FloatSlider(value=6.0, min=1.0, max=12.0, step=0.5, description='Node 2 x-coordinate:', style={'description_width': 'initial'})
y2 = 0.

x3 = widgets.FloatSlider(value=19.0, min=13.0, max=25.0, step=0.5, description='Node 3 x-coordinate:', style={'description_width': 'initial'})
y3 = 0.

x4 = widgets.FloatSlider(value=12.0, min=1.0, max=25.0, step=0.5, description='Node 4 x-coordinate:', style={'description_width': 'initial'})
y4 = widgets.FloatSlider(value=8.0, min=1.0, max=16.0, step=0.5, description='Node 4 y-coordinate:', style={'description_width': 'initial'})

# Cross sectional Area of elements
A1 = widgets.FloatSlider(value=10.0, min=1.0, max=20.0, step=0.5, description='Element 1 Area:', style={'description_width': 'initial'})
A2 = widgets.FloatSlider(value=10.0, min=1.0, max=20.0, step=0.5, description='Element 2 Area:', style={'description_width': 'initial'})
A3 = widgets.FloatSlider(value=10.0, min=1.0, max=20.0, step=0.5, description='Element 3 Area:', style={'description_width': 'initial'})

# Loads
Px = widgets.FloatSlider(value=100.0, min=1.0, max=200.0, step=1.0, description='x-dir load:', style={'description_width': 'initial'})
Py = widgets.FloatSlider(value=200.0, min=1.0, max=400.0, step=1.0, description='y-dir load:', style={'description_width': 'initial'})

# Define modulus of elasticity (in ksi)
materials = {"Timber": {"E": 1200}, "Steel": {"E": 29000}}

material_dropdown = widgets.Dropdown(options=["Timber", "Steel"], value="Timber", description='Material:')

# Display all widgets
# display(x2, x3, x4, y4, A1, A2, A3, Px, Py, material_dropdown)

# Create file path for data output
if not os.path.exists('Data'):
    os.mkdir('Data')

# Opensees Analysis with interactive plot

def update_plot(x2, x3, x4, y4, A1, A2, A3, Px, Py, material):
    # Wipe model
    ops.wipe()

    # Create ModelBuilder (2 dimensions, 3 DOFs/nodes)
    ops.model('basic', '-ndm', 2, '-ndf', 3)

    # Extract modulus of elasticity from the selected material
    E = materials[material]["E"]

    # Define materials
    ops.uniaxialMaterial("Elastic", 1, E)

    # Create nodes
    ops.node(1, x1, y1)
    ops.node(2, x2, y2)
    ops.node(3, x3, y3)
    ops.node(4, x4, y4)

    # Set boundary condition
    ops.fix(1, 1, 1, 1)
    ops.fix(2, 1, 1, 1)
    ops.fix(3, 1, 1, 1)
    ops.fix(4, 0, 0, 1)

    # Define elements
    ops.element("Truss", 1, 1, 4, A1, 1)
    ops.element("Truss", 2, 2, 4, A2, 1)
    ops.element("Truss", 3, 3, 4, A3, 1)

    # Apply loads
    ops.timeSeries("Linear", 1)
    ops.pattern("Plain", 1, 1)
    ops.load(4, Px, Py, 0)

    # Create SOE
    ops.system("BandSPD")

    # Create DOF number
    ops.numberer("RCM")

    # Create constraint handler
    ops.constraints("Plain")

    # Create integrator
    ops.integrator("LoadControl", 1.0)

    # Create algorithm
    ops.algorithm("Newton")

    # Create analysis object
    ops.analysis("Static")

    # Perform the analysis
    ops.analyze(1)

    # Get displacements
    x_disp = ops.nodeDisp(4, 1)
    y_disp = ops.nodeDisp(4, 2)

    # Clear the previous plot
    clear_output(wait=True)

    # Plotting the model
    opsv.plot_model()
    plt.title(f'Plot showing nodes and elements of model\nNode 4 Displacement: x = {x_disp:.4f} m, y = {y_disp:.4f} m')
    plt.show()

# Use interact to create interactive plot
widgets.interact(update_plot, x2=x2, x3=x3, x4=x4, y4=y4, A1=A1, A2=A2, A3=A3, Px=Px, Py=Py, material=material_dropdown)